In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import geopandas as gpd
import numpy as np
import pandas as pd
import xarray
import matplotlib.pyplot as plt
from shapely.geometry import box
from scipy.signal import savgol_filter
import rasterio
from rasterio.features import rasterize
from osgeo import gdal
import rioxarray

from func_processing import ts_telcirkel_per_jaar, bare_soil_calc, bare_soil_format


## Settings


In [ ]:
vi_str = 'bsi' # van welke vegetatie-index analyse doen?

output_folder = 'Q:/Projects/PRJ_MBAG/4d_bwk/project-telcirkels/rasters-telcirkels-mas' #folder location of the rasters

path_2022 = 'Q:/Projects/PRJ_MBAG/4d_bwk/project-telcirkels/landbouwgebruikspercelen_cut_bo_2022_clip.gpkg'
path_2023 = 'Q:/Projects/PRJ_MBAG/4d_bwk/project-telcirkels/landbouwgebruikspercelen_cut_bo_2023_clip.gpkg'
path_2024 = 'Q:/Projects/PRJ_MBAG/4d_bwk/project-telcirkels/landbouwgebruikspercelen_cut_bo_2024_clip.gpkg'


In [ ]:

#---- 
# Reading in all the parcels from Z-/Q-schijf and combining into one 
gdf_2022 = gpd.read_file(path_2022).to_crs(32631)
gdf_2023 = gpd.read_file(path_2023).to_crs(32631)
gdf_2024 = gpd.read_file(path_2024).to_crs(32631)

#----

# Dictionary met de geodataframes per jaar (aanpassen als je meer of minder jaren wilt, moet natuurlijk binnen het tijdsbereik waarvoor er rasters gedownload zijn)
analysis_years_dict = {2022: gdf_2022, 2023: gdf_2023, 2024: gdf_2024}

# Dictionary met de telperiodes (hard-coded...)
periods = {
'R1_2022': ('2022-04-01', '2022-04-20'),
'R2_2022': ('2022-04-21', '2022-05-10'),
'R3_2022': ('2022-05-11', '2022-06-10'),
'R4_2022': ('2022-06-21', '2022-07-15'),
'R1_2023': ('2023-04-01', '2023-04-20'),
'R2_2023': ('2023-04-21', '2023-05-10'),
'R3_2023': ('2023-05-11', '2023-06-10'),
'R4_2023': ('2023-06-21', '2023-07-15'),
'R1_2024': ('2024-04-01', '2024-04-20'),
'R2_2024': ('2024-04-21', '2024-05-10'),
'R3_2024': ('2024-05-11', '2024-06-10'),
'R4_2024': ('2024-06-21', '2024-07-15')}

## Bare soil analysis
Eerst de weggeschreven rasters terug inladen op de juiste manier (time index valt onder andere weg blijkbaar). Daarna op de ingeladen data de *bare soil analysis* doen. De analyse op de weggeschreven rasters doen gaat veel sneller dan het rechtstreeks uit te voeren wanneer de data van earth engine komt! De *bare soil analysis* moet per jaar apart gebeuren met de bijbehorende landbouwgebruikspercelen, terwijl de rasters gedownload zijn voor verschillenden jaren tegelijk. In deze analyse moet dus andere polygonen gebruikt worden afhankelijk van het jaar.

In [ ]:
gdf_2022.head(5)


# CRS should be 4326
gdf_2022.loc[(gdf_2022.pointid == 'Lm_11070')].plot(color='lightblue', edgecolor='black', linewidth=1)

In [ ]:
#-- Aggregation of time series over polygons
# Create empty dataframe for storage results (polygon-aggregated time series)
df_ts = pd.DataFrame(columns=['REF_ID', 'pointid', 'time', vi_str])

raster_files = [file for file in os.listdir(output_folder) if (vi_str in file) and (file.endswith('_sg.tif'))]

for file in raster_files:
    print(file)
    
    # Open the raster file
    raster = rioxarray.open_rasterio(os.path.join(output_folder,file))

    # Extract dates from the 'long_name' attribute, convert to datetime & assign to 'band' coordinate
    dates = [raster.attrs['long_name'][i] for i in range(len(raster.band))]
    time_index = pd.to_datetime(dates)
    raster['band'] = time_index


    # Xarray settings
    raster = raster.rename({'band': 'time', 'x' : 'X', 'y': 'Y'}).transpose('time', 'X', 'Y')
    raster = raster.rio.set_spatial_dims('X','Y')
    raster.rio.write_crs("EPSG:32631", inplace=True)

    # Extract pointid
    # pointid moet overeenkomen met de raster waarvoor de data opgehaald wordt, anders probleem met geen overlap tussen raster en polygon
    parts = file.split('_')
    pointid = parts[3] + '_' + parts[4]
        
    # In deze for-loop worden de tijdsreeksen voor het telpunt per jaar apart toegevoegd; afhankelijk van de geodataframes die in de dictionary zitten
    for year, gdf_year in analysis_years_dict.items():
        df_ts = ts_telcirkel_per_jaar(raster=raster, gdf=gdf_year, year=year, pointid=pointid, df_ts=df_ts)

df_ts = df_ts.sort_values(by = ['pointid','REF_ID','time']).reset_index(drop=True)
df_ts = df_ts[['pointid', 'REF_ID', 'time', vi_str]]
df_ts.to_csv(f'../outputs/output_analysis/time_series_mas_{vi_str}.csv')

df_ts

In [22]:
# Do bare soil calculation on every year seperately
df_ts = pd.read_csv(f'../outputs/output_analysis/time_series_mas_{vi_str}.csv', index_col = 0)
df_ts.time = pd.to_datetime(df_ts.time)

for year, gdf_year in analysis_years_dict.items():
    # Formatting before bare soil calculation
    # This format & the calculation should be changed if it turns out that it is too heavy for processing
    gdf_ts_year = bare_soil_format(df_ts, gdf_year, year, vi_str)
    
    # Bare soil calculation
    result = bare_soil_calc(gdf_ts_year, vi_str, periods)

    result.to_csv(f'../outputs/output_analysis/bare_soil_analysis_mas_{vi_str}_{year}_v1.csv')

result.head(5)
#gdf_ts[['OIDN', 'pointid', 'date', 'ndvi','geometry']].head(3)
#gdf.loc[~gdf.OIDN.isin(gdf_ts.OIDN.unique().tolist())].to_file('../outputs/empty_geometries.gpkg')

,pointid,year,period,% bare soil
0,DM_176.6,2024,R1,73.672736
1,Km_10019,2024,R1,4.693628
2,Km_10056.3,2024,R1,1.544639
3,Km_10183.4,2024,R1,13.982222
4,Km_10910,2024,R1,3.170508
